In [1]:
import requests

In [2]:
query = 'sum(increase(http_request_duration_seconds_bucket{path="/ping", code="200"}[10m])) by (le)'
resp = requests.get('http://localhost:9090/api/v1/query', params={'query': query})
data = resp.json()
data

{'data': {'result': [{'metric': {'le': '0.5'},
    'value': [1511418800.623, '7303.6174024464035']},
   {'metric': {'le': '5'}, 'value': [1511418800.623, '7484.631590365571']},
   {'metric': {'le': '7.5'}, 'value': [1511418800.623, '7487.682391285557']},
   {'metric': {'le': '0.25'}, 'value': [1511418800.623, '7028.0283860076715']},
   {'metric': {'le': '+Inf'}, 'value': [1511418800.623, '7487.682391285557']},
   {'metric': {'le': '0.05'}, 'value': [1511418800.623, '5259.580786055806']},
   {'metric': {'le': '0.005'}, 'value': [1511418800.623, '1120.660871274845']},
   {'metric': {'le': '0.01'}, 'value': [1511418800.623, '2172.1702550300083']},
   {'metric': {'le': '0.075'},
    'value': [1511418800.623, '5900.2489792528595']},
   {'metric': {'le': '0.025'}, 'value': [1511418800.623, '3936.550120421892']},
   {'metric': {'le': '0.1'}, 'value': [1511418800.623, '6295.83616521104']},
   {'metric': {'le': '0.75'}, 'value': [1511418800.623, '7394.124496405987']},
   {'metric': {'le': '10'}

In [3]:
buckets = sorted([(float(m['metric']['le']), float(m['value'][1])) for m in data['data']['result']])
buckets

[(0.005, 1120.660871274845),
 (0.01, 2172.1702550300083),
 (0.025, 3936.550120421892),
 (0.05, 5259.580786055806),
 (0.075, 5900.2489792528595),
 (0.1, 6295.83616521104),
 (0.25, 7028.0283860076715),
 (0.5, 7303.6174024464035),
 (0.75, 7394.124496405987),
 (1.0, 7432.76797472581),
 (2.5, 7474.4622539656175),
 (5.0, 7484.631590365571),
 (7.5, 7487.682391285557),
 (10.0, 7487.682391285557),
 (inf, 7487.682391285557)]

In [4]:
bucket_counts = []
for idx, bucket in enumerate(buckets):
    if idx > 0:
        count = bucket[1] - buckets[idx - 1][1]
    else:
        count = bucket[1]
    bucket_counts.append((bucket[0], count))
bucket_counts

[(0.005, 1120.660871274845),
 (0.01, 1051.5093837551633),
 (0.025, 1764.3798653918839),
 (0.05, 1323.030665633914),
 (0.075, 640.6681931970534),
 (0.1, 395.58718595818027),
 (0.25, 732.1922207966318),
 (0.5, 275.58901643873196),
 (0.75, 90.50709395958347),
 (1.0, 38.64347831982286),
 (2.5, 41.694279239807656),
 (5.0, 10.169336399953863),
 (7.5, 3.0508009199857042),
 (10.0, 0.0),
 (inf, 0.0)]

In [5]:
max_width = 80
block_value = max(b[1] for b in bucket_counts) / max_width
prev_bound = 0.0
for bound, count in bucket_counts:
    bar = int(count / block_value) * '▇'
    if count > 0 and not bar:
        bar = '▏' # tiny block to indicate non-zero value
    print('{:6.3f} - {:6.3f}: {}'.format(prev_bound, bound, bar))
    prev_bound = bound

 0.000 -  0.005: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.005 -  0.010: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.010 -  0.025: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.025 -  0.050: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.050 -  0.075: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.075 -  0.100: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.100 -  0.250: ▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇
 0.250 -  0.500: ▇▇▇▇▇▇▇▇▇▇▇▇
 0.500 -  0.750: ▇▇▇▇
 0.750 -  1.000: ▇
 1.000 -  2.500: ▇
 2.500 -  5.000: ▏
 5.000 -  7.500: ▏
 7.500 - 10.000: 
10.000 -    inf: 
